##### Tech Challenge 4 - Gustavo Molina Figueiredo RM 359124

### Detecção facial, análise de expressões emocionais e detecção de atividades em vídeos

Maiores explicações do código constam no relatório.

In [1]:
import cv2
import mediapipe as mp
import os
from tqdm import tqdm
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from deepface import DeepFace
import numpy as np
from collections import deque

base_options = python.BaseOptions(model_asset_path="pose_landmarker_full.task")
options = vision.PoseLandmarkerOptions(
    base_options=base_options,
    running_mode=vision.RunningMode.VIDEO,
    num_poses=1,
    min_pose_detection_confidence=0.9,
    min_pose_presence_confidence=0.9,
    min_tracking_confidence=0.9
)

detector = vision.PoseLandmarker.create_from_options(options)

def is_right_arm_up(landmarks):
    """Verifica se o braço direito está levantado acima do ombro."""
    return landmarks[14].y < landmarks[12].y

def is_left_arm_up(landmarks):
    """Verifica se o braço esquerdo está levantado acima do ombro."""
    return landmarks[13].y < landmarks[11].y

def is_hands_on_face(landmarks):
    """Verifica se uma ou ambas as mãos estão na altura do rosto."""
    if any(l is None or l.y == 0.0 for l in [landmarks[0], landmarks[4], landmarks[9], landmarks[12], landmarks[19], landmarks[20]]):
        return False
    
    shoulder_mouth_y = landmarks[12].y + ((landmarks[9].y - landmarks[12].y) / 2)
    forehead_y = landmarks[0].y - (landmarks[4].y - landmarks[0].y)  # Estima a posição da testa
    left_hand, right_hand = landmarks[19], landmarks[20]

    return ((forehead_y < left_hand.y < shoulder_mouth_y) or (forehead_y < right_hand.y < shoulder_mouth_y))

def is_hand_on_chest(landmarks):
    """Verifica se uma ou ambas as mãos estão na altura do peito."""
    required_landmarks = [11, 12, 19, 20, 23, 24]
    if any(landmarks[i] is None for i in required_landmarks):
        return False

    shoulder_y1, shoulder_y2 = landmarks[11].y, landmarks[12].y  # Ombros
    hip_y1, hip_y2 = landmarks[23].y, landmarks[24].y  # Quadris

    shoulder_y = (shoulder_y1 + shoulder_y2) / 2  
    hip_y = (hip_y1 + hip_y2) / 2  
    upper_limit_y = shoulder_y + (hip_y - shoulder_y) / 6  
    upper_torso_y = upper_limit_y + (hip_y - upper_limit_y) / 3  
    shoulder_hip_alignment = abs(shoulder_y1 - shoulder_y2) < 0.07 and abs(hip_y1 - hip_y2) < 0.07 
    left_hand, right_hand = landmarks[19], landmarks[20]

    if shoulder_hip_alignment:
        left_shoulder_x, right_shoulder_x = landmarks[11].x, landmarks[12].x
        chest_x_min, chest_x_max = min(left_shoulder_x, right_shoulder_x), max(left_shoulder_x, right_shoulder_x)

        return ((upper_limit_y <= left_hand.y < upper_torso_y and chest_x_min <= left_hand.x <= chest_x_max) or
                (upper_limit_y <= right_hand.y < upper_torso_y and chest_x_min <= right_hand.x <= chest_x_max))
    
    else:
        return ((upper_limit_y <= left_hand.y < upper_torso_y) or
                (upper_limit_y <= right_hand.y < upper_torso_y))

def is_face_sideways(landmarks):
    """Verifica se o rosto está virado de lado baseado na posição do nariz e olhos."""
    nose, left_eye, right_eye = landmarks[0], landmarks[2], landmarks[5]
    
    eyes_overlap = False
    nose_misaligned = abs(nose.x - (left_eye.x + right_eye.x) / 2) > 0.04 

    return eyes_overlap or nose_misaligned

def get_valid_landmark(landmarks, index1, index2):
    """Retorna o primeiro ponto válido entre os dois índices."""
    if landmarks[index1].visibility > 0.5:
        return landmarks[index1]
    elif landmarks[index2].visibility > 0.5:
        return landmarks[index2]
    return None 

def is_sitting(landmarks):
    """Verifica se a pessoa está sentada comparando a altura do quadril e dos joelhos."""
    hip = get_valid_landmark(landmarks, 23, 24)  
    knee = get_valid_landmark(landmarks, 25, 26)

    if not hip or not knee:
        return False  

    return abs(hip.y - knee.y) < 0.2

def is_standing(landmarks):
    """Verifica se a pessoa está de pé comparando a altura e o alinhamento lateral do quadril e joelhos."""
    hip = get_valid_landmark(landmarks, 23, 24) 
    knee = get_valid_landmark(landmarks, 25, 26)  

    if not hip or not knee:
        return False 

    hip_higher_than_knee = hip.y + 0.2 < knee.y

    x_aligned = abs(hip.x - knee.x) < 0.2 if hip and knee else True

    return hip_higher_than_knee and x_aligned

def is_arm_bent(landmarks, margin=0.2):
    """Verifica se qualquer braço está dobrado com base na posição do ombro, cotovelo e pulso."""
    
    right_shoulder = get_valid_landmark(landmarks, 11, 12)
    right_elbow = get_valid_landmark(landmarks, 13, 14)
    right_wrist = get_valid_landmark(landmarks, 15, 16)

    def check_arm(shoulder, elbow, wrist):
        """Verifica se um braço específico está dobrado."""
        if not shoulder or not elbow or not wrist:
            return False  

        shoulder_elbow_vertical = abs(shoulder.x - elbow.x) < margin
        elbow_wrist_horizontal = abs(elbow.y - wrist.y) < margin
        condition_1 = shoulder_elbow_vertical and elbow_wrist_horizontal

        shoulder_elbow_horizontal = abs(shoulder.y - elbow.y) < margin
        elbow_wrist_vertical = abs(elbow.x - wrist.x) < margin
        condition_2 = shoulder_elbow_horizontal and elbow_wrist_vertical

        return condition_1 or condition_2

    return check_arm(right_shoulder, right_elbow, right_wrist)

last_emotions = {}

emotion_history = {}

def recognize_emotions(original_frame, emotions_count, history_limit=50):
    """Detecta emoções em um frame e conta apenas mudanças reais na emoção dominante."""
    global emotion_history
    result = DeepFace.analyze(original_frame, actions=['emotion'], enforce_detection=False)

    height, width, _ = original_frame.shape

    for face in result:
        x, y, w, h = face['region']['x'], face['region']['y'], face['region']['w'], face['region']['h']
        dominant_emotion = face['dominant_emotion']

        # Normaliza a posição do rosto para evitar pequenas variações no ID
        face_id = (round(x / width, 2), round(y / height, 2))

        if face_id not in emotion_history:
            emotion_history[face_id] = deque(maxlen=history_limit)

        # Verifica se a emoção já foi detectada recentemente
        if len(emotion_history[face_id]) == 0 or emotion_history[face_id][-1] != dominant_emotion:
            emotions_count[dominant_emotion] = emotions_count.get(dominant_emotion, 0) + 1
            emotion_history[face_id].append(dominant_emotion)

        # Desenha a detecção no frame
        cv2.rectangle(original_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(original_frame, dominant_emotion, (x, y + h + 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)

    return original_frame

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

def detect_anomalies(frame, prev_landmarks, threshold=0.35):
    """Detecta movimentos bruscos baseado na variação das coordenadas das articulações."""
    
    global anomalies_count
    
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)
    
    if results.pose_landmarks:
        current_landmarks = np.array([[lm.x, lm.y] for lm in results.pose_landmarks.landmark])
        
        if prev_landmarks is not None:
            movement_diff = np.linalg.norm(current_landmarks - prev_landmarks, axis=1).mean()
            if movement_diff > threshold:
                anomalies_count += 1
        
        return frame, current_landmarks
    return frame, prev_landmarks

anomalies_count = 0

def detect_pose_and_count_movements(video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Erro ao abrir o vídeo.")
        return

    width, height = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
    
    movements_count = {
        "levantar_braco_direito": 0,
        "levantar_braco_esquerdo": 0,
        "levantar_mao(s)_na_altura_do_rosto": 0,
        "levantar_mao(s)_na_altura_do_peito": 0,
        "rosto_de_lado": 0,
        "pessoa_sentada": 0,
        "pessoa_em_pe": 0,
        "braco_dobrado": 0
    }

    emotions_count = {}

    previous_state = {key: False for key in movements_count.keys()}

    prev_landmarks = None

    for frame_index in tqdm(range(total_frames), desc="Processando vídeo"):
        ret, original_frame = cap.read()
        if not ret:
            break
       
        # Reconhecer emoções a cada n frames
        if frame_index % 5 == 0:
            frame_emotions = recognize_emotions(original_frame, emotions_count)
            frame, prev_landmarks = detect_anomalies(frame_emotions, prev_landmarks)
        else:
            frame = original_frame
       
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)
        detection_result = detector.detect_for_video(mp_image, frame_index)

        if detection_result.pose_landmarks:
            landmarks = detection_result.pose_landmarks[0]
            states = {
                "levantar_braco_direito": is_right_arm_up(landmarks),
                "levantar_braco_esquerdo": is_left_arm_up(landmarks),
                "levantar_mao(s)_na_altura_do_rosto": is_hands_on_face(landmarks),
                "levantar_mao(s)_na_altura_do_peito": is_hand_on_chest(landmarks),
                "rosto_de_lado": is_face_sideways(landmarks),
                "pessoa_sentada": is_sitting(landmarks),
                "pessoa_em_pe": is_standing(landmarks),
                "braco_dobrado": is_arm_bent(landmarks)
            }
            
            for key in states:
                if states[key] and not previous_state[key]:
                    movements_count[key] += 1
                previous_state[key] = states[key]

        # Exibir contagem de movimentos no vídeo
        y_offset = 40
        for key, value in movements_count.items():
            cv2.putText(frame, f'{key.replace("_", " ").capitalize()}: {value}', (10, y_offset),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (128, 0, 255), 1, cv2.LINE_AA)
            y_offset += 20

        # Exibir contagem de emoções no vídeo
        for key, value in emotions_count.items():
            cv2.putText(frame, f'{key.capitalize()}: {value}', (width - 200, y_offset),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1, cv2.LINE_AA)
            y_offset += 20
            
        # Exibir contagem de frames processados
        cv2.putText(frame, f'Frame: {frame_index + 1}/{total_frames}', (10, 20),
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)
        
        # Exibir contagem de anomalias detectadas
        cv2.putText(frame, f'Anomalias: {anomalies_count}', (10, y_offset),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2, cv2.LINE_AA)

        out.write(frame)
        cv2.imshow('Video', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    
    print("Contagem final de movimentos:", movements_count)
    print("Contagem final de emoções:", emotions_count)

script_dir = os.getcwd()
input_video_path = os.path.join(script_dir, 'video.mp4')
output_video_path = os.path.join(script_dir, 'output_video.mp4')

detect_pose_and_count_movements(input_video_path, output_video_path)

Processando vídeo: 100%|██████████| 3326/3326 [09:12<00:00,  6.02it/s]

Contagem final de movimentos: {'levantar_braco_direito': 2, 'levantar_braco_esquerdo': 3, 'levantar_mao(s)_na_altura_do_rosto': 10, 'levantar_mao(s)_na_altura_do_peito': 8, 'rosto_de_lado': 1, 'pessoa_sentada': 7, 'pessoa_em_pe': 4, 'braco_dobrado': 9}
Contagem final de emoções: {'sad': 48, 'fear': 47, 'happy': 98, 'angry': 11, 'neutral': 79, 'surprise': 21, 'disgust': 1}
